## Carga de datos

In [ ]:
import pandas as pd

df_productos = pd.read_pickle("../data/curated/shopify_data_to_index.pkl")
df_productos['id_producto'] = df_productos['id_producto'].astype(int)
df_productos.head(2)

In [ ]:
df_productos.iloc[0].to_dict()

## Core / Retriever

## 0. Obtencion de datos a indexar

In [ ]:
# Lista para almacenar los documentos de estadísticos
estadisticos_docs = []

# Función auxiliar para calcular estadísticas de precios
def calcular_estadisticas_precio(precios):
    precios_validos = precios.dropna()
    if len(precios_validos) == 0:
        return 0.0, 0.0, 0.0, 0.0
    
    return (
        float(precios_validos.mean()),
        float(precios_validos.min()),
        float(precios_validos.max()),
        float(precios_validos.median())
    )

# --- 1. Estadísticas Generales del Catálogo ---
total_productos = len(df_productos)
total_marcas_unicas = df_productos['marca_original'].nunique()
total_categorias = df_productos['categoria'].nunique()
total_subcategorias = df_productos['subcategoria'].nunique()

estadisticos_docs.append({
    "id": "stats_generales_catalogo",
    "tipo_estadistico": "generales_catalogo",
    "total_productos": total_productos,
    "total_marcas_unicas": total_marcas_unicas,
    "total_categorias": total_categorias,
    "total_subcategorias": total_subcategorias,
    "descripcion_estadistico": f"Estadísticas generales del catálogo: {total_productos} productos, {total_marcas_unicas} marcas, {total_categorias} categorías principales, {total_subcategorias} subcategorías.",
    "fecha_generacion": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
})

# --- 2. Estadísticas por Marca Original ---
for marca in df_productos['marca_original'].unique():
    df_marca = df_productos[df_productos['marca_original'] == marca]
    total_productos_marca = len(df_marca)
    
    # Estadísticas por categoría dentro de la marca
    for categoria in df_marca['categoria'].unique():
        df_marca_cat = df_marca[df_marca['categoria'] == categoria]
        precios = df_marca_cat['precio']
        precio_prom, precio_min, precio_max, precio_med = calcular_estadisticas_precio(precios)
        
        estadisticos_docs.append({
            "id": f"stats_marca_{marca.lower().replace(' ', '_')}_cat_{categoria.lower().replace(' ', '_')}",
            "tipo_estadistico": "marca_categoria",
            "marca": marca,
            "categoria": categoria,
            "total_productos": len(df_marca_cat),
            "precio_promedio": precio_prom,
            "precio_min": precio_min,
            "precio_max": precio_max,
            "precio_mediana": precio_med,
            "descripcion_estadistico": f"Estadísticas de {marca} en categoría {categoria}: {len(df_marca_cat)} productos, precio promedio ${precio_prom:,.0f}, rango ${precio_min:,.0f} - ${precio_max:,.0f}.",
            "fecha_generacion": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
        })
    
    # Estadísticas por tipo de repuesto dentro de la marca
    for tipo_rep in df_marca['tipo_repuesto'].unique():
        df_marca_tipo = df_marca[df_marca['tipo_repuesto'] == tipo_rep]
        precios = df_marca_tipo['precio']
        precio_prom, precio_min, precio_max, precio_med = calcular_estadisticas_precio(precios)
        
        estadisticos_docs.append({
            "id": f"stats_marca_{marca.lower().replace(' ', '_')}_tipo_{tipo_rep.lower().replace(' ', '_')}",
            "tipo_estadistico": "marca_tipo_repuesto",
            "marca": marca,
            "tipo_repuesto": tipo_rep,
            "total_productos": len(df_marca_tipo),
            "precio_promedio": precio_prom,
            "precio_min": precio_min,
            "precio_max": precio_max,
            "precio_mediana": precio_med,
            "descripcion_estadistico": f"Estadísticas de {marca} tipo {tipo_rep}: {len(df_marca_tipo)} productos, precio promedio ${precio_prom:,.0f}, rango ${precio_min:,.0f} - ${precio_max:,.0f}.",
            "fecha_generacion": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
        })

# --- 3. Estadísticas por Categoría-Subcategoría ---
for categoria in df_productos['categoria'].unique():
    df_cat = df_productos[df_productos['categoria'] == categoria]
    
    for subcategoria in df_cat['subcategoria'].unique():
        df_cat_sub = df_cat[df_cat['subcategoria'] == subcategoria]
        precios = df_cat_sub['precio']
        precio_prom, precio_min, precio_max, precio_med = calcular_estadisticas_precio(precios)
        
        estadisticos_docs.append({
            "id": f"stats_cat_{categoria.lower().replace(' ', '_')}_sub_{subcategoria.lower().replace(' ', '_')}",
            "tipo_estadistico": "categoria_subcategoria",
            "categoria": categoria,
            "subcategoria": subcategoria,
            "total_productos": len(df_cat_sub),
            "precio_promedio": precio_prom,
            "precio_min": precio_min,
            "precio_max": precio_max,
            "precio_mediana": precio_med,
            "descripcion_estadistico": f"Estadísticas de {categoria} - {subcategoria}: {len(df_cat_sub)} productos, precio promedio ${precio_prom:,.0f}, rango ${precio_min:,.0f} - ${precio_max:,.0f}.",
            "fecha_generacion": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
        })

# --- 4. Estadísticas por Tipo de Repuesto ---
for tipo_rep in df_productos['tipo_repuesto'].unique():
    df_tipo = df_productos[df_productos['tipo_repuesto'] == tipo_rep]
    precios = df_tipo['precio']
    precio_prom, precio_min, precio_max, precio_med = calcular_estadisticas_precio(precios)
    
    estadisticos_docs.append({
        "id": f"stats_tipo_{tipo_rep.lower().replace(' ', '_')}",
        "tipo_estadistico": "tipo_repuesto",
        "tipo_repuesto": tipo_rep,
        "total_productos": len(df_tipo),
        "precio_promedio": precio_prom,
        "precio_min": precio_min,
        "precio_max": precio_max,
        "precio_mediana": precio_med,
        "descripcion_estadistico": f"Estadísticas de tipo {tipo_rep}: {len(df_tipo)} productos, precio promedio ${precio_prom:,.0f}, rango ${precio_min:,.0f} - ${precio_max:,.0f}.",
        "fecha_generacion": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
    })

# --- 5. Estadísticas por Marca-Modelo (cuando hay modelos disponibles) ---
# Solo para productos que tienen modelo identificado
df_con_modelo = df_productos[df_productos['modelo'].notna() & (df_productos['modelo'] != '')]
if len(df_con_modelo) > 0:
    for marca in df_con_modelo['marca_original'].unique():
        df_marca_modelo = df_con_modelo[df_con_modelo['marca_original'] == marca]
        
        for modelo in df_marca_modelo['modelo'].unique():
            df_marca_mod = df_marca_modelo[df_marca_modelo['modelo'] == modelo]
            precios = df_marca_mod['precio']
            precio_prom, precio_min, precio_max, precio_med = calcular_estadisticas_precio(precios)
            
            estadisticos_docs.append({
                "id": f"stats_marca_{marca.lower().replace(' ', '_')}_modelo_{modelo.lower().replace(' ', '_').replace('/', '_')}",
                "tipo_estadistico": "marca_modelo",
                "marca": marca,
                "modelo": modelo,
                "total_productos": len(df_marca_mod),
                "precio_promedio": precio_prom,
                "precio_min": precio_min,
                "precio_max": precio_max,
                "precio_mediana": precio_med,
                "descripcion_estadistico": f"Estadísticas de {marca} modelo {modelo}: {len(df_marca_mod)} productos, precio promedio ${precio_prom:,.0f}, rango ${precio_min:,.0f} - ${precio_max:,.0f}.",
                "fecha_generacion": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
            })

# --- 6. Estadísticas Especiales para Llantas ---
df_llantas = df_productos[df_productos['es_llanta'] == True]
if len(df_llantas) > 0:
    precios_llantas = df_llantas['precio']
    precio_prom, precio_min, precio_max, precio_med = calcular_estadisticas_precio(precios_llantas)
    
    estadisticos_docs.append({
        "id": "stats_especial_llantas",
        "tipo_estadistico": "especial_llantas",
        "total_productos": len(df_llantas),
        "precio_promedio": precio_prom,
        "precio_min": precio_min,
        "precio_max": precio_max,
        "precio_mediana": precio_med,
        "descripcion_estadistico": f"Estadísticas especiales de llantas: {len(df_llantas)} productos, precio promedio ${precio_prom:,.0f}, rango ${precio_min:,.0f} - ${precio_max:,.0f}.",
        "fecha_generacion": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
    })

# Crear DataFrame final de estadísticos
df_estadisticos = pd.DataFrame(estadisticos_docs)
df_estadisticos

## 1. Configuración del Sistema de Retriever (`RetrieverConfig`)

El corazón de nuestro sistema de recuperación es la clase `RetrieverConfig`, ubicada en `src/core/retriever/config.py`. Esta clase `dataclass` centraliza todas las configuraciones necesarias para interactuar con Qdrant, generar embeddings y gestionar la indexación y búsqueda.

A continuación, se detallan los parámetros clave que puedes configurar:

-   **`qdrant_url`**: (str, por defecto `"http://localhost:6333"`)
    La URL de tu instancia de Qdrant. Es fundamental para que el sistema se conecte a la base de datos vectorial.

-   **`collection_name`**: (str, por defecto `"repuesto_motos_mundibot"`)
    El nombre de la colección en Qdrant con la que interactuará el sistema. Cada colección es independiente y puede tener diferentes configuraciones de vectores e índices.

-   **`dense_vector_name`**: (str, por defecto `"dense"`)
    El nombre que se le dará al campo de vectores densos dentro de la colección de Qdrant. Si se establece como una cadena vacía (`""`), los vectores densos no se crearán ni se usarán.

-   **`dense_vector_size`**: (int, por defecto `1536`)
    La dimensionalidad de los vectores densos. Para el modelo `text-embedding-3-small` de OpenAI, es `1536`.

-   **`dense_distance`**: (`qdrant_client.models.Distance`, por defecto `models.Distance.COSINE`)
    La métrica de distancia utilizada para comparar vectores densos. Comúnmente `COSINE` para embeddings de OpenAI.

-   **`sparse_vector_name`**: (str, por defecto `"sparse"`)
    El nombre que se le dará al campo de vectores dispersos. Si se establece como una cadena vacía (`""`), los vectores dispersos no se crearán ni se usarán.

-   **`openai_model`**: (str, por defecto `"text-embedding-3-small"`)
    El nombre del modelo de OpenAI utilizado para generar embeddings densos.

-   **`openai_api_key`**: (Optional[str], por defecto `None`)
    Tu clave de API de OpenAI. Si no se proporciona, el sistema intentará cargarla desde la variable de entorno `OPENAI_API_KEY` (usando `python-dotenv`).

-   **`sparse_model`**: (str, por defecto `"prithivida/Splade_PP_en_v1"`)
    El nombre del modelo utilizado por FastEmbed para generar embeddings dispersos (por ejemplo, un modelo SPLADE).

-   **`batch_size`**: (int, por defecto `100`)
    El número de documentos a procesar en cada lote durante la indexación. Afecta el rendimiento y el uso de memoria.

-   **`max_concurrent_requests`**: (int, por defecto `10`)
    El número máximo de solicitudes concurrentes a la API de OpenAI o a Qdrant durante la generación de embeddings o la indexación.

-   **`payload_indices`**: (Dict[str, `qdrant_client.models.PayloadSchemaType`])
    Un diccionario que define los campos del payload para los cuales se crearán índices en Qdrant. Esto es crucial para realizar filtrados eficientes. Los tipos de esquema incluyen `KEYWORD`, `TEXT`, `INTEGER`, `FLOAT`, `BOOL`, etc.

Al entender estos parámetros, puedes adaptar el sistema a diversas necesidades, desde colecciones de búsqueda híbrida hasta colecciones puramente informativas o de metadatos.

In [ ]:
from src.core.retriever.config import RetrieverConfig
from qdrant_client import models
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# --- 1. Configuración para una Colección Informativa (sin vectores) ---
# Queremos una colección que almacene estadísticos, no documentos para búsqueda semántica.
# Por lo tanto, deshabilitamos la creación de vectores densos y dispersos.
informative_config = RetrieverConfig(
    qdrant_url="http://localhost:6333",
    collection_name="repuestos_estadisticos_mundibot", # Un nuevo nombre para esta colección
    dense_vector_name="",  # Deshabilita la creación de vectores densos
    sparse_vector_name="", # Deshabilita la creación de vectores dispersos
    
    # Definimos los índices de payload relevantes para nuestros estadísticos
    payload_indices={
        "id": models.PayloadSchemaType.KEYWORD,
        "tipo_estadistico": models.PayloadSchemaType.KEYWORD,
        "marca": models.PayloadSchemaType.KEYWORD,
        "categoria": models.PayloadSchemaType.KEYWORD,
        "subcategoria": models.PayloadSchemaType.KEYWORD,
        "modelo": models.PayloadSchemaType.KEYWORD,
        "tipo_repuesto": models.PayloadSchemaType.KEYWORD,
        "total_productos": models.PayloadSchemaType.INTEGER,
        "precio_promedio": models.PayloadSchemaType.FLOAT,
        "precio_min": models.PayloadSchemaType.FLOAT,
        "precio_max": models.PayloadSchemaType.FLOAT,
        "precio_mediana": models.PayloadSchemaType.FLOAT,
        "descripcion_estadistico": models.PayloadSchemaType.TEXT,
        "fecha_generacion": models.PayloadSchemaType.KEYWORD,
    }
)

print("✅ Configuración genérica para estadísticos creada:")
print(f"  📁 Colección: {informative_config.collection_name}")
print(f"  🚫 Vectores densos: {'HABILITADOS' if informative_config.dense_vector_name else 'DESHABILITADOS'}")
print(f"  🚫 Vectores dispersos: {'HABILITADOS' if informative_config.sparse_vector_name else 'DESHABILITADOS'}")
print(f"  🗂️  Índices de payload: {len(informative_config.payload_indices)} campos")
print(f"  📋 Campos indexados: {list(informative_config.payload_indices.keys())}")

In [ ]:
from src.core.retriever import create_estadisticos_config, IndexingConfig

# Crear configuración específica para estadísticos usando la función helper
informative_config = create_estadisticos_config(
    qdrant_url="http://localhost:6333",
    collection_name="repuestos_estadisticos_mundibot"
)

print("✅ Configuración genérica para estadísticos creada:")
print(f"  📁 Colección: {informative_config.collection_name}")
print(f"  🚫 Vectores densos: {'HABILITADOS' if informative_config.dense_vector_name else 'DESHABILITADOS'}")
print(f"  🚫 Vectores dispersos: {'HABILITADOS' if informative_config.sparse_vector_name else 'DESHABILITADOS'}")
print(f"  🗂️  Índices de payload: {len(informative_config.payload_indices)} campos")
print(f"  📋 Campos indexados: {list(informative_config.payload_indices.keys())}")

## 2. Configuracion de la indexacion

In [ ]:
from src.core.retriever import IndexingConfig

# Crear configuración de indexación para estadísticos
estadisticos_indexing_config = IndexingConfig(
    batch_size=500,
    max_concurrent_batches=3,
    id_field="id",  # Campo que contiene el ID del documento
    upsert_mode=True,  # Permite actualizar documentos existentes
    store_original_id=True  # Guarda el ID original en el payload
)

print("✅ Configuración creada:")
print(f"  📁 Colección: {informative_config.collection_name}")
print(f"  🚫 Vectores densos: {'HABILITADOS' if informative_config.dense_vector_name else 'DESHABILITADOS'}")
print(f"  🚫 Vectores dispersos: {'HABILITADOS' if informative_config.sparse_vector_name else 'DESHABILITADOS'}")
print(f"  🗂️  Índices de payload: {len(informative_config.payload_indices)} campos")
print(f"  🆔 Campo ID: {estadisticos_indexing_config.id_field}")
print(f"  🔄 Modo upsert: {'SÍ' if estadisticos_indexing_config.upsert_mode else 'NO'}")

## 3 Indexacion

In [ ]:
from src.core.retriever.utils import RetrieverSystemManager

# --- Indexación de Documentos ---
logger.info("🚀 Iniciando indexación de documentos estadísticos...")

# Inicializar sistema
system_manager = RetrieverSystemManager(informative_config)

# Setup del sistema
setup_stats = await system_manager.setup_system(overwrite=True)
logger.info(f"✅ Setup completado: {setup_stats}")

# Función de callback para progreso
def progress_callback(processed, total):
    percentage = (processed / total) * 100
    logger.info(f"📊 Progreso: {processed}/{total} ({percentage:.1f}%)")

# Verificar estructura de documentos
ejemplo_doc = df_estadisticos.iloc[0].to_dict()
print(f"🔍 Verificando estructura de documentos:")
print(f"   📊 Total documentos: {len(df_estadisticos)}")
print(f"   🔑 Campos disponibles: {list(ejemplo_doc.keys())}")
print(f"   🆔 Campo ID ({estadisticos_indexing_config.id_field}): {ejemplo_doc.get(estadisticos_indexing_config.id_field)}")
print(f"   🔄 Modo upsert: {'SÍ' if estadisticos_indexing_config.upsert_mode else 'NO'}")

# Indexar documentos
await system_manager.indexer.index_documents(
    documents=df_estadisticos.to_dict(orient='records'),
    indexing_config=estadisticos_indexing_config,
    progress_callback=progress_callback
)

logger.info("✅ Indexación completada!")

# Verificar resultados
total_indexed = system_manager.collection_manager.count_points(exact=False)
print(f"📈 Documentos indexados: {total_indexed}")
print(f"📊 Documentos esperados: {len(df_estadisticos)}")
print(f"✅ Éxito: {total_indexed == len(df_estadisticos)}")

## 🔍 Ejemplos de Consultas con Filtros

Ahora que tenemos los documentos estadísticos indexados, vamos a demostrar diferentes tipos de consultas usando solo filtros (sin vectores). Esta colección está optimizada para búsquedas por metadatos y filtros específicos.

### Tipos de consultas disponibles:
1. **Búsqueda básica con filtros simples**
2. **Filtros por tipo de estadístico**
3. **Filtros por marca y modelo**
4. **Filtros por rangos de precios**
5. **Combinación de múltiples filtros**
6. **Búsqueda por ID original**

###  Verificar Conexión y Estructura

In [ ]:
from src.core.retriever import create_estadisticos_config

informative_config = create_estadisticos_config(
    qdrant_url="http://localhost:6333",
    collection_name="repuestos_estadisticos_mundibot"
)

# --- Verificar Sistema y Datos Indexados ---
print("🔍 Verificando sistema y datos indexados...")

# Verificar conexión
collection_info = system_manager.collection_manager.get_collection_info()
print(f"📊 Información de la colección:")
print(f"   📁 Nombre: {system_manager.config.collection_name}")
print(f"   📈 Total puntos: {collection_info.points_count}")
print(f"   🗂️ Índices de payload: {len(collection_info.payload_schema)}")
print(f"   🟢 Estado: {collection_info.status.value}")
print(f"   📦 Segmentos: {collection_info.segments_count}")

# Mostrar algunos índices de payload
print(f"\n🗂️ Índices de payload disponibles:")
for field_name, field_info in list(collection_info.payload_schema.items())[:8]:
    print(f"   📋 {field_name}: {field_info.data_type.value} ({field_info.points} puntos)")
if len(collection_info.payload_schema) > 8:
    print(f"   ... y {len(collection_info.payload_schema) - 8} campos más")

# Verificar algunos documentos usando search_with_filters
print(f"\n🔍 Verificando estructura de documentos indexados...")
sample_results = await system_manager.retriever.search_with_filters(
    query="",  # Query vacía para búsqueda solo por filtros
    limit=3    # ✅ Ahora limit está en el lugar correcto
)

print(f"📄 Documentos de muestra encontrados: {len(sample_results.results)}")
for i, result in enumerate(sample_results.results[:2]):
    print(f"\n📋 Documento {i+1}:")
    print(f"   🆔 ID Original: {result.payload.get('_original_id', 'N/A')}")
    print(f"   📊 Tipo: {result.payload.get('tipo_estadistico', 'N/A')}")
    print(f"   🏷️ Marca: {result.payload.get('marca', 'N/A')}")
    print(f"   🔢 Total Productos: {result.payload.get('total_productos', 'N/A')}")
    print(f"   💰 Precio Promedio: ${result.payload.get('precio_promedio', 0):,.0f}")

**Alternativa usando SearchConfig:**

Si prefieres usar el método search() directamente:

In [ ]:
from src.core.retriever.config import SearchConfig

# Verificar algunos documentos usando SearchConfig
print(f"\n🔍 Verificando estructura de documentos indexados...")

# Crear configuración de búsqueda
search_config = SearchConfig(limit=3)

sample_results = await system_manager.retriever.search(
    query="",  # Query vacía
    search_config=search_config
)

print(f"📄 Documentos de muestra encontrados: {len(sample_results.results)}")
for i, result in enumerate(sample_results.results[:2]):
    print(f"\n📋 Documento {i+1}:")
    print(f"   🆔 ID Original: {result.payload.get('_original_id', 'N/A')}")
    print(f"   📊 Tipo: {result.payload.get('tipo_estadistico', 'N/A')}")
    print(f"   🏷️ Marca: {result.payload.get('marca', 'N/A')}")
    print(f"   🔢 Total Productos: {result.payload.get('total_productos', 'N/A')}")
    print(f"   💰 Precio Promedio: ${result.payload.get('precio_promedio', 0):,.0f}")

### Búsquedas por Tipo de Estadístico

In [ ]:
# 1. Buscar estadísticas generales  
print("1️⃣ Estadísticas generales del catálogo:")

# Crear filtro personalizado para tipo_estadistico
from qdrant_client import models

tipo_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="tipo_estadistico",
            match=models.MatchValue(value="generales_catalogo")
        )
    ]
)

general_results = await system_manager.retriever.search_with_filters(
    query="estadísticas generales",
    filters=tipo_filter,  # ✅ es un objeto Filter
    limit=5
)


print(f"📄 Documentos de muestra encontrados: {len(general_results.results)}")
for i, result in enumerate(general_results.results[:2]):
    print(f"\n📋 Documento {i+1}:")
    print(f"   🆔 ID Original: {result.payload.get('_original_id', 'N/A')}")
    print(f"   📊 Tipo: {result.payload.get('tipo_estadistico', 'N/A')}")
    print(f"   🏷️ Marca: {result.payload.get('marca', 'N/A')}")
    print(f"   🔢 Total Productos: {result.payload.get('total_productos', 'N/A')}")

### Función Helper para Crear Filtros Fácilmente:

In [ ]:
# --- Función Helper para Filtros ---
from qdrant_client import models

def create_simple_filter(field: str, value: str) -> models.Filter:
    """Crear un filtro simple para un campo específico."""
    return models.Filter(
        must=[
            models.FieldCondition(
                key=field,
                match=models.MatchValue(value=value)
            )
        ]
    )

def create_range_filter(field: str, min_val: float = None, max_val: float = None) -> models.Filter:
    """Crear un filtro de rango para precios."""
    range_condition = models.Range()
    if min_val is not None:
        range_condition.gte = min_val
    if max_val is not None:
        range_condition.lte = max_val
    
    return models.Filter(
        must=[
            models.FieldCondition(
                key=field,
                range=range_condition
            )
        ]
    )

print("✅ Funciones helper para filtros creadas")

In [ ]:
# --- Búsquedas por Tipo de Estadístico 
print("📊 Ejemplos de búsqueda por tipo de estadístico...\n")

# 1. Buscar estadísticas generales
print("1️⃣ Estadísticas generales del catálogo:")
general_filter = create_simple_filter("tipo_estadistico", "generales_catalogo")
general_results = await system_manager.retriever.search_with_filters(
    query="estadísticas generales",
    filters=general_filter,
    limit=5
)

print(f"   📈 Encontrados: {len(general_results.results)} resultados")
for result in general_results.results:
    payload = result.payload
    print(f"   📄 {payload.get('descripcion_estadistico', 'Sin descripción')[:80]}...")

# 2. Buscar estadísticas por marca
print(f"\n2️⃣ Estadísticas por marca:")
marca_filter = create_simple_filter("tipo_estadistico", "marca_modelo")
marca_results = await system_manager.retriever.search_with_filters(
    query="marca",
    filters=marca_filter,
    limit=10
)

print(f"   📈 Encontrados: {len(marca_results.results)} resultados")
for result in marca_results.results[:5]:
    marca = result.payload.get('marca', 'N/A')
    total = result.payload.get('total_productos', 0)
    print(f"   🏷️ {marca}: {total} productos")

# 3. Buscar por marca específica (Honda)
print(f"\n3️⃣ Estadísticas de HONDA:")
honda_filter = create_simple_filter("marca", "HONDA")
honda_results = await system_manager.retriever.search_with_filters(
    query="Honda",
    filters=honda_filter,
    limit=8
)

print(f"   📈 Encontrados: {len(honda_results.results)} resultados para HONDA")
for result in honda_results.results[:5]:
    payload = result.payload
    tipo = payload.get('tipo_estadistico', 'N/A')
    total = payload.get('total_productos', 0)
    descripcion = payload.get('descripcion_estadistico', 'Sin descripción')
    print(f"   📊 {tipo}: {total} productos - {descripcion[:50]}...")

# 4. Buscar por rangos de precio
print(f"\n4️⃣ Productos con precios altos (>100,000):")
precio_filter = create_range_filter("precio_promedio", min_val=100000)
precio_results = await system_manager.retriever.search_with_filters(
    query="precio alto",
    filters=precio_filter,
    limit=8
)

print(f"   📈 Encontrados: {len(precio_results.results)} resultados")
for result in precio_results.results[:5]:
    payload = result.payload
    marca = payload.get('marca', 'N/A')
    precio_prom = payload.get('precio_promedio', 0)
    precio_max = payload.get('precio_max', 0)
    print(f"   💰 {marca}: Promedio ${precio_prom:,.0f}, Máximo ${precio_max:,.0f}")